## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from IPython.display import display, HTML
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


**Identify Target and Feature Variables**

In [2]:
# Identify the target variable
target_variable = "IS_SUCCESSFUL"

# Identify the feature variables
feature_variables = application_df.columns.tolist()
feature_variables.remove(target_variable)

# Create a DataFrame
data = {
    "Variable Type": ["Target Variable"] + ["Feature Variable"] * len(feature_variables),
    "Variable Name": [target_variable] + feature_variables
}

df = pd.DataFrame(data)
df

,Variable Type,Variable Name
0,Target Variable,IS_SUCCESSFUL
1,Feature Variable,EIN
2,Feature Variable,NAME
3,Feature Variable,APPLICATION_TYPE
4,Feature Variable,AFFILIATION
5,Feature Variable,CLASSIFICATION
6,Feature Variable,USE_CASE
7,Feature Variable,ORGANIZATION
8,Feature Variable,STATUS
9,Feature Variable,INCOME_AMT


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN"])

In [4]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

# Display the unique counts
print(unique_counts)

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
# Print the formatted output
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Choose a cutoff value for binning
cutoff_value = 500

# Create a list of application types to be replaced with "Other"
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [7]:
# Look at CLASSIFICATION value counts for binning
# Check the value counts of the CLASSIFICATION column
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
filtered_counts = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(filtered_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose a cutoff value for binning
cutoff_value = 1000

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [11]:
names_to_replace = list(name_counts[name_counts < 100].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(
        name, "Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [12]:
application_df.dtypes

NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [13]:
categorical_list = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
categorical_list

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [15]:
# Split our preprocessed data into our features and target arrays
# Define the target variable
y = application_df_encoded["IS_SUCCESSFUL"].values

# Define the feature variables
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting arrays to verify the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 74)
X_test shape: (6860, 74)
y_train shape: (27439,)
y_test shape: (6860,)


In [16]:
# Perform one-hot encoding for the target variable
from keras.utils import to_categorical
y_train_enc = to_categorical(y_train)
y_test_enc = to_categorical(y_test)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features_total = len(X_train[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               9600      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 16)                1040      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 18913 (73.88 KB)
Trainable params: 18913 (73.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
858/858 [==============================] - 3s 2ms/step - loss: 0.5085 - accuracy: 0.7465
Epoch 2/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4921 - accuracy: 0.7576
Epoch 3/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4890 - accuracy: 0.7587
Epoch 4/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4870 - accuracy: 0.7595
Epoch 5/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4854 - accuracy: 0.7586
Epoch 6/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4850 - accuracy: 0.7607
Epoch 7/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4832 - accuracy: 0.7606
Epoch 8/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4833 - accuracy: 0.7595
Epoch 9/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4822 - accuracy: 0.7629
Epoch 10/50
858/858 [==============================] - 1s 2ms/step - loss: 0.4819 - accuracy: 0.7625

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5291 - accuracy: 0.7496 - 417ms/epoch - 2ms/step
Loss: 0.5290688872337341, Accuracy: 0.749562680721283


In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features_total = len(X_train[0])
hidden_nodes_layer1 = 512
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 32

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation = "relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=2, activation="softmax"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               38400     
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 32)                4128      
                                                                 
 dense_7 (Dense)             (None, 2)                 66        
                                                                 
Total params: 108258 (422.88 KB)
Trainable params: 108258 (422.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model
nn2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled,y_train_enc,epochs=50)

Epoch 1/50
858/858 [==============================] - 3s 2ms/step - loss: 0.5088 - accuracy: 0.7487
Epoch 2/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4936 - accuracy: 0.7567
Epoch 3/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4897 - accuracy: 0.7599
Epoch 4/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4886 - accuracy: 0.7610
Epoch 5/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4863 - accuracy: 0.7621
Epoch 6/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4862 - accuracy: 0.7614
Epoch 7/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4852 - accuracy: 0.7616
Epoch 8/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4842 - accuracy: 0.7619
Epoch 9/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4833 - accuracy: 0.7626
Epoch 10/50
858/858 [==============================] - 2s 2ms/step - loss: 0.4833 - accuracy: 0.7630

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test_enc,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5363 - accuracy: 0.7496 - 408ms/epoch - 2ms/step
Loss: 0.5362971425056458, Accuracy: 0.749562680721283


In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features_total = len(X_train[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 16

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation = "relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               9600      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 16)                1040      
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 18913 (73.88 KB)
Trainable params: 18913 (73.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
from tensorflow.keras.optimizers import Adam

# Create an Adam optimizer with a custom learning rate
custom_lr = 0.001  # Set your desired learning rate here
custom_optimizer = Adam(learning_rate=custom_lr)

# Compile the model with the custom optimizer
nn3.compile(loss="binary_crossentropy", optimizer=custom_optimizer, metrics=["accuracy"])

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    "best_model_weights.h5",
    monitor="loss",
    verbose=1,
    save_best_only=True,
    mode="min",
    save_weights_only=True,
    save_freq="epoch"
)

In [29]:
# Train the model
fit_model3 = nn3.fit(X_train_scaled,y_train,epochs=50,callbacks=[checkpoint],batch_size=64)

Epoch 1/50
401/429 [===========================>..] - ETA: 0s - loss: 0.5094 - accuracy: 0.7460
Epoch 1: loss improved from inf to 0.50812, saving model to best_model_weights.h5
429/429 [==============================] - 2s 2ms/step - loss: 0.5081 - accuracy: 0.7463
Epoch 2/50
404/429 [===========================>..] - ETA: 0s - loss: 0.4923 - accuracy: 0.7569
Epoch 2: loss improved from 0.50812 to 0.49163, saving model to best_model_weights.h5
429/429 [==============================] - 1s 2ms/step - loss: 0.4916 - accuracy: 0.7575
Epoch 3/50
403/429 [===========================>..] - ETA: 0s - loss: 0.4876 - accuracy: 0.7584
Epoch 3: loss improved from 0.49163 to 0.48825, saving model to best_model_weights.h5
429/429 [==============================] - 1s 2ms/step - loss: 0.4883 - accuracy: 0.7582
Epoch 4/50
411/429 [===========================>..] - ETA: 0s - loss: 0.4869 - accuracy: 0.7584
Epoch 4: loss improved from 0.48825 to 0.48637, saving model to best_model_weights.h5
429/429 [

In [30]:
nn3.load_weights("./best_model_weights.h5")

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5093 - accuracy: 0.7507 - 378ms/epoch - 2ms/step
Loss: 0.5093197822570801, Accuracy: 0.7507288455963135


In [32]:
# Export our model to HDF5 file
filepath = "AlphabetSoupCharity_Optimization.h5"
nn.save(filepath, save_format='h5')

c:\Users\Eally\anaconda3\envs\dev_class\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
from tensorflow.keras.models import load_model

# load the trained model from an HDF5 file
nn=load_model("./AlphabetSoupCharity_Optimization.h5")

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5291 - accuracy: 0.7496 - 365ms/epoch - 2ms/step
Loss: 0.5290688872337341, Accuracy: 0.749562680721283
